This is not ideal I think but this is to slice all the delta data into the 8 different types... then just test the existing model on the different categories separately? Later, I'll make models on the different categories separately..... I just feel bad since I'm about to use up so much space making multiple datasets omg but okay

In [1]:
#imports
import uproot as ur
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import vector as v

import atlas_mpl_style as ampl #makes matplotlib follow atlas formal style
ampl.use_atlas_style()

v.register_awkward()

data_path = '/fast_scratch/atlas_images/v01-45/'

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2" #specify GPU
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

# energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split

from sklearn.metrics import roc_auc_score, roc_curve

import graph_util as gu
import plot_util as pu
import matplotlib.pyplot as plt

2021-07-16 07:22:09.668664: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import convert_sets_splitdelta

In [3]:
events = ur.open(data_path+'delta_medium.root')

tree = events['EventTree']

In [4]:
idbranch = tree.arrays('truthPartPdgId')
truth_ID = idbranch.truthPartPdgId

In [5]:
#just need to get the indices of each of them - I should probably just do this with like np.where or smtn but for now this works heheheh :(
ID1114 = []
IDn2214 = []    
IDn2114 = []    
IDn2224 = []   
ID2214 = []
ID2224 = []
IDn1114 = []
ID2114 = []
for i in range(len(truth_ID)):
    if (-2114 in truth_ID[i]): 
        IDn2114.append(i)
    elif (-2224 in truth_ID[i]):
        IDn2224.append(i)
    elif (2214 in truth_ID[i]): 
        ID2214.append(i)
    elif (2224 in truth_ID[i]):
        ID2224.append(i)
    elif (-1114 in truth_ID[i]): 
        IDn1114.append(i)
    elif (2114 in truth_ID[i]):
        ID2114.append(i)
    elif (1114 in truth_ID[i]): 
        ID1114.append(i)
    elif (-2214 in truth_ID[i]):
        IDn2214.append(i)

#this is literally still so janky but whatever
flag = np.zeros(len(truth_ID))
flag[IDn2214] = -2214
flag[ID2214] = 2214
flag[ID1114] = 1114
flag[IDn1114] = -1114
flag[ID2114] = 2114
flag[IDn2114] = -2114
flag[ID2224] = 2224
flag[IDn2224] = -2224

In [6]:
#CHOOSE:

#then choose the mask from there: eg
mask = flag == 2214

In [7]:
convert_sets_splitdelta.doit(mask, data_path+'delta2214.npz')

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
Normalizing


/home/kaelaekd/start_tf/LCStudies/classifier/convert_sets_splitdelta.py:61: RuntimeWarning: divide by zero encountered in log
  cell_e = np.nan_to_num(np.log(cell_e), posinf = 0, neginf=0)


Writing out
Done! /fast_scratch/atlas_images/v01-45/delta2214.npz


In [8]:
mask = flag== -2214
convert_sets_splitdelta.doit(mask, data_path+'delta-2214.npz')

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
Normalizing
Writing out
Done! /fast_scratch/atlas_images/v01-45/delta-2214.npz


In [9]:
mask = flag == 1114
convert_sets_splitdelta.doit(mask, data_path+'delta1114.npz')

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
Normalizing
Writing out
Done! /fast_scratch/atlas_images/v01-45/delta1114.npz


In [10]:
mask = flag == -1114
convert_sets_splitdelta.doit(mask, data_path+'delta-1114.npz')

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
Normalizing
Writing out
Done! /fast_scratch/atlas_images/v01-45/delta-1114.npz


In [11]:
mask = flag == 2114
convert_sets_splitdelta.doit(mask, data_path+'delta2114.npz')

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
Normalizing
Writing out
Done! /fast_scratch/atlas_images/v01-45/delta2114.npz


In [12]:
mask = flag == -2114
convert_sets_splitdelta.doit(mask, data_path+'delta-2114.npz')

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
Normalizing
Writing out
Done! /fast_scratch/atlas_images/v01-45/delta-2114.npz


In [13]:
mask = flag == 2224
convert_sets_splitdelta.doit(mask, data_path+'delta2224.npz')

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
Normalizing
Writing out
Done! /fast_scratch/atlas_images/v01-45/delta2224.npz


In [14]:
mask = flag == -2224
convert_sets_splitdelta.doit(mask, data_path+'delta-2224.npz')

Working on /fast_scratch/atlas_images/v01-45/delta_medium.root
Loading data
Normalizing
Writing out
Done! /fast_scratch/atlas_images/v01-45/delta-2224.npz


In [15]:
delt2214 = np.load(data_path+'delta2214.npz')
deltn2214 = np.load(data_path+'delta-2214.npz')
delt1114 = np.load(data_path+'delta1114.npz')
deltn1114 = np.load(data_path+'delta-1114.npz')
delt2114 = np.load(data_path+'delta2114.npz')
deltn2114 = np.load(data_path+'delta-2114.npz')
delt2224 = np.load(data_path+'delta2224.npz')
deltn2224 = np.load(data_path+'delta-2224.npz')

In [16]:
x = delt2214['X']
x2 = delt1114['X']
x3 = deltn2224['X']

In [18]:
print(len(x), len(x2), len(x3))

77131 82681 86737


In [19]:
#extract data... one by one... sigh
X = delt2214['X']
clus_eta=delt2214['clus_eta']
clus_pt = delt2214['clus_pt']
clus_E = delt2214['clus_e']
clus_et = delt2214['clus_e_t']

print(len(X))

77131


In [20]:
#extract data... one by one... sigh
X2 = deltn2214['X']
clus_eta2=deltn2214['clus_eta']
clus_pt2 = deltn2214['clus_pt']
clus_E2 = deltn2214['clus_e']
clus_et2 = deltn2214['clus_e_t']

print(len(X2))

81718


In [21]:
#extract data... one by one... sigh
X3 = delt1114['X']
clus_eta3=delt1114['clus_eta']
clus_pt3 = delt1114['clus_pt']
clus_E3 = delt1114['clus_e']
clus_et3 = delt1114['clus_e_t']

print(len(X3))

82681


In [22]:
#extract data... one by one... sigh
X4 = deltn1114['X']
clus_eta4=deltn1114['clus_eta']
clus_pt4 = deltn1114['clus_pt']
clus_E4 = deltn1114['clus_e']
clus_et4 = deltn1114['clus_e_t']

print(len(X4))

86864


In [23]:
#extract data... one by one... sigh
X5 = delt2114['X']
clus_eta5=delt2114['clus_eta']
clus_pt5 = delt2114['clus_pt']
clus_E5 = delt2114['clus_e']
clus_et5 = delt2114['clus_e_t']

print(len(X5))

77803


In [24]:
#extract data... one by one... sigh
X6 = deltn2114['X']
clus_eta6=deltn2114['clus_eta']
clus_pt6 = deltn2114['clus_pt']
clus_E6 = deltn2114['clus_e']
clus_et6 = deltn2114['clus_e_t']

print(len(X6))

82247


In [25]:
#extract data... one by one... sigh
X7 = delt2224['X']
clus_eta7=delt2224['clus_eta']
clus_pt7 = delt2224['clus_pt']
clus_E7 = delt2224['clus_e']
clus_et7 = delt2224['clus_e_t']

print(len(X7))

82987


In [26]:
#extract data... one by one... sigh
X8 = deltn2224['X']
clus_eta8=deltn2224['clus_eta']
clus_pt8 = deltn2224['clus_pt']
clus_E8 = deltn2224['clus_e']
clus_et8 = deltn2224['clus_e_t']

print(len(X8))

86737


In [27]:
#do them all with the same amount
def doit(dat, outfile):
    X, clus_eta, clus_pt, clus_E, clus_et = dat
    X_all = np.array(X[:77000])
    eta_all = np.array(clus_eta[:77000])
    pt_all = np.array(clus_pt[:77000])
    E_all = np.array(clus_E[:77000])
    et_all = np.array(clus_et[:77000])
    
    (X_train, X_val, X_test,  
     eta_train, eta_val, eta_test, 
     ET_train, ET_val, ET_test, 
     pt_train, pt_val, pt_test, 
     Eng_train, Eng_val, Eng_test) = data_split(X_all, eta_all, et_all, pt_all, E_all, val=10000, test=37000)
    
    np.savez(data_path+outfile, X_train, X_val, X_test, eta_train, eta_val, eta_test, ET_train, ET_val, ET_test, pt_train, pt_val, pt_test, Eng_train, Eng_val, Eng_test)

In [28]:
doit((X, clus_eta, clus_pt, clus_E, clus_et), 'nn_delta2214')

In [29]:
doit((X2, clus_eta2, clus_pt2, clus_E2, clus_et2), 'nn_delta-2214')

In [30]:
doit((X3, clus_eta3, clus_pt3, clus_E3, clus_et3), 'nn_delta1114')

In [31]:
doit((X4, clus_eta4, clus_pt4, clus_E4, clus_et4), 'nn_delta-1114')

In [32]:
doit((X5, clus_eta5, clus_pt5, clus_E5, clus_et5), 'nn_delta2114')

In [33]:
doit((X6, clus_eta6, clus_pt6, clus_E6, clus_et6), 'nn_delta-2114')

In [34]:
doit((X7, clus_eta7, clus_pt7, clus_E7, clus_et7), 'nn_delta2224')

In [35]:
doit((X8, clus_eta8, clus_pt8, clus_E8, clus_et8), 'nn_delta-2224')

Kind of icky still but that should work!!

NOTE: I think I'll keep the .root files in fast scratch since I'm not the only one accessing those, but I'm gonna move every single npz file to /data/atlas/ now!!!! okay!